In [1]:
import time
import tabulate
from scipy import stats

import sys 
sys.path.append("/Users/larry/Documents/GitHubProjects/inventory") 

from inventory.gsm_tree import *

## Introduction

This script contains examples to demonstrate the use of `gsm_tree.py,` which implements the
dynamic programming algorithm for the guaranteed-service model (GSM)
for multi-echelon inventory systems with tree structures by Graves and Willems (2000).

'node' and 'stage' are used interchangeably in the documentation.

The primary data object is the `NetworkX DiGraph`, which contains all of the data
for the GSM instance. The following attributes are used to specify input data:
* Node-level attributes
    - processing_time [T]
    - external_inbound_cst [si]
    - external_outbound_cst [s]
    - holding_cost [h]
    - demand_bound_constant [z_alpha]
    - external_demand_mean [mu]
    - external_demand_standard_deviation [sigma]
* Edge-level attributes
    - units_required (e.g., on edge i->j, units_required units of item i are
required to make 1 unit of item j)

When adding nodes using `nx.DiGraph.add_node()`, you can add attributes as
arguments to `add_node()`. Subsequently, to get or set node attributes, the node
is treated like a dict with the attributes as keys (as strings), so use
`node['holding_cost']`, etc.

(c) Lawrence V. Snyder
Lehigh University and Opex Analytics

## Instance #1: Example 6.5

In [2]:
# Create a new DiGraph object.
ex65_graph = nx.DiGraph()

# Add nodes, with the relevant attributes. Attributes are specified as
# arguments to add_node().
ex65_graph.add_node(1, processing_time=2,
					external_inbound_cst=1,
					holding_cost=1,
					demand_bound_constant=1)
ex65_graph.add_node(2, processing_time=1,
					external_outbound_cst=0,
					holding_cost=3,
					demand_bound_constant=1,
					external_demand_standard_deviation=1)
ex65_graph.add_node(3, processing_time=1,
					holding_cost=2,
					demand_bound_constant=1)
ex65_graph.add_node(4, processing_time=1,
					external_outbound_cst=1,
					holding_cost=3,
					demand_bound_constant=1,
					external_demand_standard_deviation=1)

# Add edges. (units_required is the only edge attribute, but we don't need
# it here because it equals 1 for every edge.)
ex65_graph.add_edge(1, 3)
ex65_graph.add_edge(3, 2)
ex65_graph.add_edge(3, 4)

# We can add any arbitrary attributes we want to the graph, nodes, and edges.
# Here, we'll add a label to the graph.
ex65_graph.graph['problem_name'] = 'Example 6.5 instance'

In [3]:
# We have to preprocess the graph before we can solve the model or perform
# other functions. Preprocessing calculates some intermediate quantities, fills
# in some missing attributes, etc. Note that preprocess_tree() returns a new graph,
# it does not modify the original graph. In the line below, we just replace the
# old one with the new one.
ex65_graph = preprocess_tree(ex65_graph)

# Start the timer.
start_time = time.time()

# Solve the problem.
opt_cost, opt_cst = optimize_committed_service_times(ex65_graph)

# Stop the timer.
end_time = time.time()

In [4]:
# Get some other quantities based on the solution.
SI = inbound_cst(ex65_graph, ex65_graph.nodes, opt_cst)
nlt = net_lead_time(ex65_graph, ex65_graph.nodes, opt_cst)
safety_stock = safety_stock_levels(ex65_graph, ex65_graph.nodes, opt_cst)
base_stock = base_stock_levels(ex65_graph, ex65_graph.nodes, opt_cst)

# Display the results.
print('\nSolved {:s} in {:.4f} seconds.'.format(ex65_graph.graph['problem_name'],
											  end_time - start_time))
print('Optimal cost: {:.4f}'.format(opt_cost))
print('Optimal solution:\n')
results = []
for k in ex65_graph.nodes:
	results.append([k, opt_cst[k], SI[k], nlt[k], safety_stock[k], base_stock[k]])
print(tabulate.tabulate(results, headers=['Stage', 'S', 'SI', 'NLT', 'Safety Stock', 'Base-Stock Level']))


Solved Example 6.5 instance in 0.0013 seconds.
Optimal cost: 8.2779
Optimal solution:

  Stage    S    SI    NLT    Safety Stock    Base-Stock Level
-------  ---  ----  -----  --------------  ------------------
      1    0     1      3         2.44949             2.44949
      2    0     0      1         1                   1
      3    0     0      1         1.41421             1.41421
      4    1     0      0         0                   0


## Instance #2: Figure 6.14

In [5]:
# Create DiGraph.
fig614_graph = nx.DiGraph()

# Add nodes.
fig614_graph.add_node('Raw_Material', processing_time=2,
							  holding_cost=0.01)
fig614_graph.add_node('Process_Wafers', processing_time=3,
							  holding_cost=0.03)
fig614_graph.add_node('Package_Test_Wafers', processing_time=2,
							  holding_cost=0.04)
fig614_graph.add_node('Imager_Base', processing_time=4,
							  holding_cost=0.06)
fig614_graph.add_node('Imager_Assembly', processing_time=2,
							  holding_cost=0.12)
fig614_graph.add_node('Ship_to_Final_Assembly', processing_time=3,
							  holding_cost=0.13)
fig614_graph.add_node('Camera', processing_time=6,
							  holding_cost=0.20)
fig614_graph.add_node('Circuit_Board', processing_time=4,
							  holding_cost=0.08)
fig614_graph.add_node('Other_Parts', processing_time=3,
							  holding_cost=0.04)
fig614_graph.add_node('Build_Test_Pack', processing_time=2,
							  holding_cost=0.50,
							  external_outbound_cst=2,
							  external_demand_standard_deviation=10,
							  demand_bound_constant=stats.norm.ppf(0.95))

# Add edges.
fig614_graph.add_edge('Raw_Material', 'Process_Wafers')
fig614_graph.add_edge('Process_Wafers', 'Package_Test_Wafers')
fig614_graph.add_edge('Package_Test_Wafers', 'Imager_Assembly')
fig614_graph.add_edge('Imager_Base', 'Imager_Assembly')
fig614_graph.add_edge('Imager_Assembly', 'Ship_to_Final_Assembly')
fig614_graph.add_edge('Camera', 'Build_Test_Pack')
fig614_graph.add_edge('Ship_to_Final_Assembly', 'Build_Test_Pack')
fig614_graph.add_edge('Circuit_Board', 'Build_Test_Pack')
fig614_graph.add_edge('Other_Parts', 'Build_Test_Pack')

# Add problem name.
fig614_graph.graph['problem_name'] = 'Figure 6.14 instance'

In [6]:
# Preprocess and solve.
fig614_graph = preprocess_tree(fig614_graph)
start_time = time.time()
opt_cost, opt_cst = optimize_committed_service_times(fig614_graph)
end_time = time.time()

In [7]:
# Get other outputs.
SI = inbound_cst(fig614_graph, fig614_graph.nodes, opt_cst)
nlt = net_lead_time(fig614_graph, fig614_graph.nodes, opt_cst)
safety_stock = safety_stock_levels(fig614_graph, fig614_graph.nodes, opt_cst)
base_stock = base_stock_levels(fig614_graph, fig614_graph.nodes, opt_cst)

# Display the results.
print('\nSolved {:s} in {:.4f} seconds.'.format(fig614_graph.graph['problem_name'],
											  end_time - start_time))
print('Optimal cost: {:.4f}'.format(opt_cost))
print('Optimal solution:\n')
results = []
for k in fig614_graph.nodes:
	results.append([k, opt_cst[k], SI[k], nlt[k], safety_stock[k], base_stock[k]])
print(tabulate.tabulate(results, headers=['Stage', 'S', 'SI', 'NLT', 'Safety Stock', 'Base-Stock Level']))


Solved Figure 6.14 instance in 0.0072 seconds.
Optimal cost: 18.8240
Optimal solution:

Stage                     S    SI    NLT    Safety Stock    Base-Stock Level
----------------------  ---  ----  -----  --------------  ------------------
Raw_Material              0     0      2         23.2617             23.2617
Process_Wafers            3     0      0          0                   0
Package_Test_Wafers       5     3      0          0                   0
Imager_Base               4     0      0          0                   0
Imager_Assembly           7     5      0          0                   0
Ship_to_Final_Assembly    0     7     10         52.0148             52.0148
Camera                    0     0      6         40.2905             40.2905
Circuit_Board             0     0      4         32.8971             32.8971
Other_Parts               0     0      3         28.4897             28.4897
Build_Test_Pack           2     0      0          0                   0


## Instance #3: Yinan's counterexample

In [8]:
# Create DiGraph.
yinan_graph = nx.DiGraph()

# Add nodes.
yinan_graph.add_node(1, processing_time=4,
					external_inbound_cst=0,
					holding_cost=1,
					demand_bound_constant=1)
yinan_graph.add_node(2, processing_time=0,
					holding_cost=1.1,
					demand_bound_constant=1,
					external_outbound_cst=0,
					external_demand_standard_deviation=1)
yinan_graph.add_node(3, processing_time=2,
					holding_cost=1,
					demand_bound_constant=1)
yinan_graph.add_node(4, processing_time=0,
					holding_cost=100000,
					demand_bound_constant=1,
					external_outbound_cst=0,
					external_demand_standard_deviation=1)

# Add edges.
yinan_graph.add_edge(1, 2)
yinan_graph.add_edge(3, 2)
yinan_graph.add_edge(3, 4)

# Add graph label.
yinan_graph.graph['problem_name'] = 'Yinan''s Counterexample'

In [9]:
# Preprocess and solve.
yinan_graph = preprocess_tree(yinan_graph)
start_time = time.time()
opt_cost, opt_cst = optimize_committed_service_times(yinan_graph)
end_time = time.time()

In [10]:
# Get other outputs.
inbound_cst = inbound_cst(yinan_graph, yinan_graph.nodes, opt_cst)
net_lead_time = net_lead_time(yinan_graph, yinan_graph.nodes, opt_cst)
safety_stock = safety_stock_levels(yinan_graph, yinan_graph.nodes, opt_cst)
base_stock = base_stock_levels(yinan_graph, yinan_graph.nodes, opt_cst)

# Display the results.
print('\nSolved {:s} in {:.4f} seconds.'.format(yinan_graph.graph['problem_name'],
											  end_time - start_time))
print('Optimal cost: {:.4f}'.format(opt_cost))
print('Optimal solution:\n')
results = []
for k in yinan_graph.nodes:
	results.append([k, opt_cst[k], inbound_cst[k], net_lead_time[k], safety_stock[k], base_stock[k]])
print(tabulate.tabulate(results, headers=['Stage', 'S', 'SI', 'NLT', 'Safety Stock', 'Base-Stock Level']))


Solved Yinans Counterexample in 0.0011 seconds.
Optimal cost: 4.0000
Optimal solution:

  Stage    S    SI    NLT    Safety Stock    Base-Stock Level
-------  ---  ----  -----  --------------  ------------------
      1    0     0      4               2                   2
      2    0     0      0               0                   0
      3    0     0      2               2                   2
      4    0     0      0               0                   0
